In [1]:
#!/usr/bin/env python3
"""
OWL-ViT Implementation for Digital Forensics
Open-World Localization with Vision Transformer for text-based evidence detection
"""

import torch
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pathlib import Path
import json
from datetime import datetime

# Installation requirements:
# pip install transformers torch pillow matplotlib

# =============================================================================
# CONFIGURATION
# =============================================================================

CONFIG = {
    # Input/Output Paths
    'input_folder': '../datasets/images/objects/raw',           # Folder with evidence images
    'output_folder': '../datasets/images/objects/detection',  # Output folder for results
    'single_image_path': "../datasets/images/objects/raw/knife_161.jpg",                               # Path for single image analysis

    # Model Configuration
    'model_name': 'google/owlvit-base-patch32',              # owlvit-base-patch32, owlvit-base-patch16, owlvit-large-patch14
    'confidence_threshold': 0.15,                           # Minimum confidence for detections
    'device': 'auto',                                       # 'auto', 'cuda', 'cpu'

    # Search Configuration
    'search_mode': 'comprehensive',                         # 'comprehensive', 'targeted', 'custom'
    'custom_queries': [],                                   # Custom search queries (for custom mode)
    'target_categories': ['weapons', 'drugs', 'money'],     # Categories to search (for targeted mode)

    # Forensic Analysis Settings
    'priority_threshold': 7.0,                             # Minimum score for high priority findings
    'enable_threat_assessment': True,                      # Enable threat level assessment
    'save_detailed_analysis': True,                       # Save detailed forensic analysis

    # Processing Options
    'batch_processing': True,                              # Enable batch processing
    'max_images': None,                                    # Maximum images to process (None = all)
    'show_progress': True,                                 # Show progress information
    'create_investigation_report': True,                   # Create comprehensive investigation report

    # Output Options
    'save_visualizations': True,                          # Save detection visualizations
    'save_json_reports': True,                            # Save JSON analysis reports
    'visualization_dpi': 300,                             # DPI for saved visualizations
    'include_confidence_scores': True,                    # Include confidence scores in outputs

    # Forensic Search Queries by Category
    'forensic_queries': {
        'weapons': [
            "gun", "pistol", "rifle", "firearm", "weapon",
            "knife", "blade", "sword", "machete", "dagger",
            "brass knuckles", "baton", "club", "taser",
            "modified weapon", "sawed-off shotgun"
        ],
        'drugs': [
            "white powder", "pills", "tablets", "capsules",
            "syringe", "needle", "drug paraphernalia", "pipe", "bong",
            "scale", "drug scale", "baggies with drugs", "marijuana",
            "cocaine powder", "heroin", "methamphetamine", "drug lab equipment"
        ],
        'money': [
            "stack of money", "cash bundle", "hundred dollar bills",
            "large amount of money", "currency", "banknotes",
            "money counting machine", "briefcase with money", "cash in bags"
        ],
        'electronics': [
            "cell phone", "smartphone", "laptop computer", "tablet",
            "hard drive", "USB drive", "memory card", "SIM card",
            "gaming console", "router", "burner phone", "encrypted device"
        ],
        'documents': [
            "passport", "driver license", "identity card", "fake ID",
            "credit card", "forged document", "bank statement",
            "legal document", "counterfeit passport", "birth certificate"
        ],
        'containers': [
            "suitcase", "duffel bag", "backpack", "briefcase",
            "safe", "lockbox", "hidden compartment", "secret stash",
            "weapon case", "drug packaging", "evidence bag"
        ],
        'tools': [
            "lock picking tools", "crowbar", "bolt cutters", "drill",
            "saw", "hacking device", "surveillance equipment",
            "wiretapping device", "burglary tools", "break-in tools"
        ],
        'vehicles': [
            "getaway car", "motorcycle", "stolen vehicle", "van",
            "truck with hidden compartment", "modified vehicle"
        ]
    },

    # Priority scoring weights for different categories
    'category_weights': {
        'weapons': 9.0,
        'drugs': 8.0,
        'money': 6.0,
        'tools': 7.0,
        'electronics': 5.0,
        'documents': 6.0,
        'containers': 4.0,
        'vehicles': 5.0
    },

    # Supported image formats
    'supported_formats': {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'}
}

class ForensicOWLViT:
    def __init__(self, config=None):
        """Initialize OWL-ViT model with configuration"""
        self.config = config or CONFIG

        # Load model and processor
        self.processor = OwlViTProcessor.from_pretrained(self.config['model_name'])
        self.model = OwlViTForObjectDetection.from_pretrained(self.config['model_name'])

        # Set device
        if self.config['device'] == 'auto':
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = self.config['device']
        self.model.to(self.device)

        # Create output directories
        self._setup_output_directories()

        if self.config['show_progress']:
            print(f"🎯 OWL-ViT Forensic Analyzer initialized")
            print(f"   Model: {self.config['model_name']}")
            print(f"   Device: {self.device}")
            print(f"   Search mode: {self.config['search_mode']}")
            print(f"   Confidence threshold: {self.config['confidence_threshold']}")

    def analyze_single_image(self, image_path=None):
        """Analyze a single image for forensic evidence"""
        image_path = image_path or self.config['single_image_path']
        if not image_path:
            raise ValueError("No image path provided")

        if self.config['show_progress']:
            print(f"🔍 Analyzing: {Path(image_path).name}")

        # Get search queries based on mode
        search_queries = self._get_search_queries()

        # Perform search
        analysis = self._search_image(image_path, search_queries)

        # Enhanced forensic analysis
        if self.config['save_detailed_analysis']:
            analysis = self._enhance_forensic_analysis(analysis)

        # Save outputs
        if self.config['save_json_reports']:
            self._save_analysis_report(analysis, single_image=True)

        if self.config['save_visualizations']:
            self._create_visualization(analysis, search_queries)

        return analysis

    def analyze_batch(self):
        """Analyze all images in the input folder"""
        input_path = Path(self.config['input_folder'])
        if not input_path.exists():
            raise FileNotFoundError(f"Input folder not found: {input_path}")

        # Get image files
        image_files = self._get_image_files(input_path)

        if self.config['max_images']:
            image_files = image_files[:self.config['max_images']]

        if self.config['show_progress']:
            print(f"📁 Processing {len(image_files)} images from {input_path}")

        batch_results = []
        high_priority_findings = []
        search_queries = self._get_search_queries()

        for i, image_file in enumerate(image_files, 1):
            if self.config['show_progress']:
                print(f"   [{i}/{len(image_files)}] {image_file.name}")

            try:
                analysis = self._analyze_image_file(image_file, search_queries)
                batch_results.append(analysis)

                # Check for high-priority findings
                if analysis.get('threat_assessment', {}).get('priority_score', 0) >= self.config['priority_threshold']:
                    high_priority_findings.append(analysis)

            except Exception as e:
                if self.config['show_progress']:
                    print(f"      ❌ Error: {e}")

        # Create comprehensive report
        investigation_report = self._create_investigation_report(batch_results, high_priority_findings, search_queries)

        # Save outputs
        if self.config['save_json_reports']:
            self._save_batch_results(batch_results, investigation_report)

        if self.config['show_progress']:
            print(f"✅ Investigation complete!")
            print(f"   Total processed: {len(batch_results)}")
            print(f"   High priority: {len(high_priority_findings)}")
            print(f"   Results saved to: {self.config['output_folder']}")

        return {
            'batch_results': batch_results,
            'investigation_report': investigation_report,
            'high_priority_findings': high_priority_findings
        }

    def custom_investigation(self, image_path, custom_queries):
        """Perform custom investigation with user-defined queries"""
        if self.config['show_progress']:
            print(f"🕵️ Custom investigation: {len(custom_queries)} search terms")
            for i, query in enumerate(custom_queries, 1):
                print(f"  {i}. '{query}'")

        analysis = self._search_image(image_path, custom_queries)
        analysis['investigation_type'] = 'custom'
        analysis['custom_queries'] = custom_queries

        if self.config['save_detailed_analysis']:
            analysis = self._enhance_forensic_analysis(analysis)

        # Save results
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        if self.config['save_json_reports']:
            filename = f"custom_investigation_{timestamp}.json"
            output_file = Path(self.config['output_folder']) / 'custom_investigations' / filename
            output_file.parent.mkdir(parents=True, exist_ok=True)
            with open(output_file, 'w') as f:
                json.dump(analysis, f, indent=2)

            if self.config['show_progress']:
                print(f"📄 Investigation saved: {output_file}")

        return analysis

    def _get_search_queries(self):
        """Get search queries based on configured mode"""
        if self.config['search_mode'] == 'custom':
            return self.config['custom_queries']

        elif self.config['search_mode'] == 'targeted':
            queries = []
            for category in self.config['target_categories']:
                if category in self.config['forensic_queries']:
                    queries.extend(self.config['forensic_queries'][category])
            return queries

        elif self.config['search_mode'] == 'comprehensive':
            queries = []
            for category_queries in self.config['forensic_queries'].values():
                queries.extend(category_queries)
            return queries

        else:
            raise ValueError(f"Unknown search mode: {self.config['search_mode']}")

    def _search_image(self, image_path, search_queries):
        """Search for specific items in image using text queries"""
        # Load and process image
        image = Image.open(image_path).convert('RGB')

        # Prepare inputs
        inputs = self.processor(text=search_queries, images=image, return_tensors="pt")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Run detection
        with torch.no_grad():
            outputs = self.model(**inputs)

        # Process results
        target_sizes = torch.Tensor([image.size[::-1]]).to(self.device)
        results = self.processor.post_process_object_detection(
            outputs=outputs,
            target_sizes=target_sizes,
            threshold=self.config['confidence_threshold']
        )

        detections = []
        for i, query in enumerate(search_queries):
            boxes = results[0]["boxes"][results[0]["labels"] == i]
            scores = results[0]["scores"][results[0]["labels"] == i]

            for box, score in zip(boxes, scores):
                if score >= self.config['confidence_threshold']:
                    detections.append({
                        'query': query,
                        'bbox': box.cpu().numpy().tolist(),
                        'confidence': float(score.cpu().numpy()),
                        'query_index': i,
                        'category': self._get_query_category(query)
                    })

        return {
            'image_path': str(image_path),
            'timestamp': datetime.now().isoformat(),
            'search_queries': search_queries,
            'search_mode': self.config['search_mode'],
            'detections': detections,
            'image_size': image.size,
            'total_detections': len(detections),
            'config_used': {
                'model_name': self.config['model_name'],
                'confidence_threshold': self.config['confidence_threshold']
            }
        }

    def _enhance_forensic_analysis(self, analysis):
        """Add enhanced forensic analysis to results"""
        detections = analysis['detections']

        # Group detections by category
        category_analysis = {}
        for detection in detections:
            category = detection['category']
            if category not in category_analysis:
                category_analysis[category] = {
                    'count': 0,
                    'max_confidence': 0,
                    'items': []
                }

            category_analysis[category]['count'] += 1
            category_analysis[category]['max_confidence'] = max(
                category_analysis[category]['max_confidence'],
                detection['confidence']
            )
            category_analysis[category]['items'].append(detection['query'])

        # Calculate threat assessment
        threat_assessment = self._calculate_threat_assessment(category_analysis)

        # Add enhanced analysis
        analysis.update({
            'category_analysis': category_analysis,
            'threat_assessment': threat_assessment,
            'evidence_summary': self._create_evidence_summary(category_analysis),
            'recommendations': self._generate_recommendations(category_analysis, threat_assessment)
        })

        return analysis

    def _calculate_threat_assessment(self, category_analysis):
        """Calculate comprehensive threat assessment"""
        priority_score = 0
        threat_indicators = []

        for category, data in category_analysis.items():
            # Base score from category weight and detection confidence
            if category in self.config['category_weights']:
                weight = self.config['category_weights'][category]
                confidence_bonus = data['max_confidence'] * 2
                count_bonus = min(data['count'] * 0.5, 2.0)  # Cap count bonus

                category_score = (weight + confidence_bonus + count_bonus)
                priority_score += category_score

                threat_indicators.append({
                    'category': category,
                    'severity': self._get_severity_level(weight),
                    'count': data['count'],
                    'confidence': data['max_confidence'],
                    'score_contribution': category_score
                })

        # Determine overall threat level
        if priority_score >= 20:
            threat_level = "CRITICAL"
        elif priority_score >= 15:
            threat_level = "HIGH"
        elif priority_score >= 8:
            threat_level = "MEDIUM"
        elif priority_score >= 3:
            threat_level = "LOW"
        else:
            threat_level = "MINIMAL"

        return {
            'threat_level': threat_level,
            'priority_score': min(priority_score, 10.0),
            'threat_indicators': threat_indicators,
            'assessment_timestamp': datetime.now().isoformat()
        }

    def _get_severity_level(self, weight):
        """Get severity level based on category weight"""
        if weight >= 8:
            return "CRITICAL"
        elif weight >= 6:
            return "HIGH"
        elif weight >= 4:
            return "MEDIUM"
        else:
            return "LOW"

    def _get_query_category(self, query):
        """Determine which category a query belongs to"""
        for category, queries in self.config['forensic_queries'].items():
            if query in queries:
                return category
        return 'unknown'

    def _create_evidence_summary(self, category_analysis):
        """Create summary of evidence found"""
        summary = {
            'categories_detected': list(category_analysis.keys()),
            'total_evidence_items': sum(data['count'] for data in category_analysis.values()),
            'highest_confidence_category': max(
                category_analysis.items(),
                key=lambda x: x[1]['max_confidence'],
                default=(None, {'max_confidence': 0})
            )[0],
            'most_items_category': max(
                category_analysis.items(),
                key=lambda x: x[1]['count'],
                default=(None, {'count': 0})
            )[0]
        }
        return summary

    def _generate_recommendations(self, category_analysis, threat_assessment):
        """Generate investigative recommendations"""
        recommendations = []

        # High-priority recommendations based on findings
        for category in category_analysis:
            if category == 'weapons':
                recommendations.append("URGENT: Secure scene and implement weapon safety protocols")
                recommendations.append("Conduct ballistics analysis and weapon tracing")
            elif category == 'drugs':
                recommendations.append("Implement evidence preservation for substance analysis")
                recommendations.append("Consider drug trafficking investigation")
            elif category == 'money':
                recommendations.append("Financial investigation recommended")
                recommendations.append("Check for money laundering indicators")
            elif category == 'electronics':
                recommendations.append("Digital forensics analysis required")
                recommendations.append("Secure devices for data extraction")

        # General recommendations based on threat level
        threat_level = threat_assessment['threat_level']
        if threat_level in ['CRITICAL', 'HIGH']:
            recommendations.append("Priority case classification recommended")
            recommendations.append("Additional resources and expertise required")
            recommendations.append("Consider multi-agency coordination")

        return recommendations

    def _analyze_image_file(self, image_file, search_queries):
        """Analyze single image file"""
        analysis = self._search_image(str(image_file), search_queries)

        if self.config['save_detailed_analysis']:
            analysis = self._enhance_forensic_analysis(analysis)

        # Save individual outputs
        if self.config['save_visualizations']:
            self._create_visualization(analysis, search_queries)

        return analysis

    def _create_visualization(self, analysis, search_queries):
        """Create and save visualization"""
        image_path = analysis['image_path']
        image = Image.open(image_path)

        fig, ax = plt.subplots(1, 1, figsize=(15, 10))
        ax.imshow(image)

        # Color mapping for different categories
        colors = {
            'weapons': 'red',
            'drugs': 'purple',
            'money': 'green',
            'electronics': 'blue',
            'documents': 'orange',
            'containers': 'brown',
            'tools': 'pink',
            'vehicles': 'cyan',
            'unknown': 'gray'
        }

        # Draw detections
        for detection in analysis['detections']:
            bbox = detection['bbox']
            query = detection['query']
            confidence = detection['confidence']
            category = detection.get('category', 'unknown')

            color = colors.get(category, 'gray')

            # Draw bounding box
            rect = patches.Rectangle(
                (bbox[0], bbox[1]),
                bbox[2] - bbox[0],
                bbox[3] - bbox[1],
                linewidth=2,
                edgecolor=color,
                facecolor='none'
            )
            ax.add_patch(rect)

            # Add label
            if self.config['include_confidence_scores']:
                label = f"{query}\n{confidence:.2f}"
            else:
                label = query

            ax.text(bbox[0], bbox[1] - 10, label,
                   bbox=dict(boxstyle="round,pad=0.3", facecolor=color, alpha=0.8),
                   fontsize=8, color='white', weight='bold')

        # Title with threat assessment if available
        title = f"OWL-ViT Forensic Analysis: {Path(image_path).name}\n"
        title += f"{len(analysis['detections'])} detections found"

        if 'threat_assessment' in analysis:
            threat = analysis['threat_assessment']
            title += f" | Threat Level: {threat['threat_level']}"
            title += f" | Priority Score: {threat['priority_score']:.1f}"

        ax.set_title(title)
        ax.axis('off')

        # Add legend
        legend_elements = [patches.Patch(color=color, label=category.title())
                          for category, color in colors.items()
                          if any(d.get('category') == category for d in analysis['detections'])]
        if legend_elements:
            ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.15, 1))

        # Save visualization
        output_path = Path(self.config['output_folder']) / 'visualizations'
        output_file = output_path / f"{Path(image_path).stem}_analysis.png"
        plt.savefig(output_file, dpi=self.config['visualization_dpi'], bbox_inches='tight')
        plt.close()

        return str(output_file)

    def _create_investigation_report(self, batch_results, high_priority_findings, search_queries):
        """Create comprehensive investigation report"""
        return {
            'investigation_summary': {
                'timestamp': datetime.now().isoformat(),
                'search_mode': self.config['search_mode'],
                'total_images_analyzed': len(batch_results),
                'high_priority_cases': len(high_priority_findings),
                'search_queries_used': len(search_queries),
                'config_used': {
                    'model_name': self.config['model_name'],
                    'confidence_threshold': self.config['confidence_threshold'],
                    'priority_threshold': self.config['priority_threshold']
                }
            },
            'evidence_statistics': self._calculate_evidence_statistics(batch_results),
            'threat_analysis': self._analyze_batch_threats(batch_results),
            'high_priority_cases': [
                {
                    'image_path': result['image_path'],
                    'threat_level': result.get('threat_assessment', {}).get('threat_level', 'UNKNOWN'),
                    'priority_score': result.get('threat_assessment', {}).get('priority_score', 0),
                    'categories_found': list(result.get('category_analysis', {}).keys()),
                    'total_detections': result['total_detections']
                }
                for result in high_priority_findings
            ],
            'search_effectiveness': self._analyze_search_effectiveness(batch_results, search_queries)
        }

    def _calculate_evidence_statistics(self, batch_results):
        """Calculate comprehensive evidence statistics"""
        category_totals = {}
        total_detections = 0
        images_with_evidence = 0

        for result in batch_results:
            total_detections += result['total_detections']
            if result['total_detections'] > 0:
                images_with_evidence += 1

            category_analysis = result.get('category_analysis', {})
            for category, data in category_analysis.items():
                if category not in category_totals:
                    category_totals[category] = {'count': 0, 'images': 0}
                category_totals[category]['count'] += data['count']
                category_totals[category]['images'] += 1

        return {
            'total_detections': total_detections,
            'images_with_evidence': images_with_evidence,
            'evidence_categories': category_totals,
            'detection_rate': images_with_evidence / len(batch_results) if batch_results else 0
        }

    def _analyze_batch_threats(self, batch_results):
        """Analyze threat levels across batch"""
        threat_levels = {}
        priority_scores = []

        for result in batch_results:
            threat_assessment = result.get('threat_assessment', {})
            level = threat_assessment.get('threat_level', 'MINIMAL')
            score = threat_assessment.get('priority_score', 0)

            threat_levels[level] = threat_levels.get(level, 0) + 1
            priority_scores.append(score)

        return {
            'threat_level_distribution': threat_levels,
            'average_priority_score': sum(priority_scores) / len(priority_scores) if priority_scores else 0,
            'max_priority_score': max(priority_scores) if priority_scores else 0,
            'critical_threat_percentage': (threat_levels.get('CRITICAL', 0) / len(batch_results)) * 100 if batch_results else 0
        }

    def _analyze_search_effectiveness(self, batch_results, search_queries):
        """Analyze effectiveness of different search queries"""
        query_performance = {}

        for result in batch_results:
            for detection in result['detections']:
                query = detection['query']
                if query not in query_performance:
                    query_performance[query] = {
                        'hits': 0,
                        'total_confidence': 0,
                        'avg_confidence': 0
                    }

                query_performance[query]['hits'] += 1
                query_performance[query]['total_confidence'] += detection['confidence']

        # Calculate averages
        for query, data in query_performance.items():
            if data['hits'] > 0:
                data['avg_confidence'] = data['total_confidence'] / data['hits']

        # Sort by effectiveness (hits * avg_confidence)
        effectiveness_scores = {
            query: data['hits'] * data['avg_confidence']
            for query, data in query_performance.items()
        }

        return {
            'query_performance': query_performance,
            'most_effective_queries': sorted(
                effectiveness_scores.items(),
                key=lambda x: x[1],
                reverse=True
            )[:10],
            'total_unique_queries_with_hits': len(query_performance),
            'query_hit_rate': len(query_performance) / len(search_queries) if search_queries else 0
        }

    def _save_analysis_report(self, analysis, single_image=False):
        """Save analysis report to JSON"""
        if single_image:
            output_path = Path(self.config['output_folder']) / 'single_analysis'
            filename = f"{Path(analysis['image_path']).stem}_analysis.json"
        else:
            output_path = Path(self.config['output_folder']) / 'individual_reports'
            filename = f"{Path(analysis['image_path']).stem}_report.json"

        output_path.mkdir(parents=True, exist_ok=True)
        output_file = output_path / filename

        with open(output_file, 'w') as f:
            json.dump(analysis, f, indent=2)

        return str(output_file)

    def _save_batch_results(self, batch_results, investigation_report):
        """Save batch processing results"""
        output_path = Path(self.config['output_folder'])

        # Save complete batch results
        batch_file = output_path / 'batch_investigation_results.json'
        with open(batch_file, 'w') as f:
            json.dump(batch_results, f, indent=2)

        # Save investigation report
        report_file = output_path / 'investigation_report.json'
        with open(report_file, 'w') as f:
            json.dump(investigation_report, f, indent=2)

        if self.config['show_progress']:
            print(f"   📄 Batch results: {batch_file}")
            print(f"   📋 Investigation report: {report_file}")

    def _get_image_files(self, input_path):
        """Get list of image files from input directory"""
        image_files = []
        for file_path in input_path.glob('*'):
            if file_path.suffix.lower() in self.config['supported_formats']:
                image_files.append(file_path)
        return sorted(image_files)

    def _setup_output_directories(self):
        """Create necessary output directories"""
        output_path = Path(self.config['output_folder'])

        directories = ['visualizations', 'individual_reports', 'single_analysis', 'custom_investigations']
        for directory in directories:
            (output_path / directory).mkdir(parents=True, exist_ok=True)

# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main execution function"""

    # Initialize detector with configuration
    detector = ForensicOWLViT(CONFIG)

    print("=== OWL-ViT Forensic Investigation System ===")
    print(f"Configuration loaded:")
    print(f"  Input folder: {CONFIG['input_folder']}")
    print(f"  Output folder: {CONFIG['output_folder']}")
    print(f"  Model: {CONFIG['model_name']}")
    print(f"  Search mode: {CONFIG['search_mode']}")
    print(f"  Confidence threshold: {CONFIG['confidence_threshold']}")

    # Choose analysis mode
    if CONFIG['single_image_path']:
        print("\n🖼️  Single Image Analysis Mode")
        try:
            result = detector.analyze_single_image()
            print(f"✅ Analysis complete!")
            print(f"   Total detections: {result['total_detections']}")
            if 'threat_assessment' in result:
                threat = result['threat_assessment']
                print(f"   Threat level: {threat['threat_level']}")
                print(f"   Priority score: {threat['priority_score']:.1f}")
            if result.get('category_analysis'):
                categories = list(result['category_analysis'].keys())
                print(f"   Evidence categories: {categories}")
        except Exception as e:
            print(f"❌ Error: {e}")

    elif CONFIG['batch_processing']:
        print("\n📁 Batch Investigation Mode")
        try:
            results = detector.analyze_batch()
            report = results['investigation_report']
            summary = report['investigation_summary']

            print(f"✅ Investigation complete!")
            print(f"   Images analyzed: {summary['total_images_analyzed']}")
            print(f"   High priority cases: {summary['high_priority_cases']}")
            print(f"   Evidence detection rate: {report['evidence_statistics']['detection_rate']:.1%}")

            # Show threat distribution
            threat_dist = report['threat_analysis']['threat_level_distribution']
            print(f"   Threat distribution: {threat_dist}")

        except Exception as e:
            print(f"❌ Error: {e}")

    elif CONFIG['search_mode'] == 'custom' and CONFIG['custom_queries']:
        print("\n🕵️ Custom Investigation Mode")
        image_path = CONFIG['single_image_path'] or input("Enter image path: ")
        try:
            result = detector.custom_investigation(image_path, CONFIG['custom_queries'])
            print(f"✅ Custom investigation complete!")
            print(f"   Detections found: {result['total_detections']}")
        except Exception as e:
            print(f"❌ Error: {e}")

    else:
        print("\n⚠️  No analysis mode configured. Please set:")
        print("     - CONFIG['single_image_path'] for single image analysis")
        print("     - CONFIG['batch_processing'] = True for batch processing")
        print("     - CONFIG['search_mode'] = 'custom' with custom_queries for custom investigation")

if __name__ == "__main__":
    main()

preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

D:\Interpreter\Python\Projects\ai_cyberforensics\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\Models\Hugging_Face\cache_root\hub\models--google--owlvit-base-patch32. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/613M [00:00<?, ?B/s]

🎯 OWL-ViT Forensic Analyzer initialized
   Model: google/owlvit-base-patch32
   Device: cpu
   Search mode: comprehensive
   Confidence threshold: 0.15
=== OWL-ViT Forensic Investigation System ===
Configuration loaded:
  Input folder: ../datasets/images/objects/raw
  Output folder: ../datasets/images/objects/detection
  Model: google/owlvit-base-patch32
  Search mode: comprehensive
  Confidence threshold: 0.15

🖼️  Single Image Analysis Mode
🔍 Analyzing: knife_161.jpg


D:\Interpreter\Python\Projects\ai_cyberforensics\.venv\Lib\site-packages\transformers\models\owlvit\processing_owlvit.py:233: FutureWarning: `post_process_object_detection` method is deprecated for OwlVitProcessor and will be removed in v5. Use `post_process_grounded_object_detection` instead.
  warnings.warn(


✅ Analysis complete!
   Total detections: 0
   Threat level: MINIMAL
   Priority score: 0.0
